In [1]:
#Lab Glassware

In [2]:
# Instalação de bibliotecas essenciais
!pip install roboflow
!pip install tensorflow
!pip install opencv-python
!pip install matplotlib
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Looking in indexes: https://download.pytorch.org/whl/cu118



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# Importação das bibliotecas
import os
import cv2
import torch
import json
import pandas as pd
import torchvision
import matplotlib.pyplot as plt
from PIL import Image
from collections import defaultdict
from torchvision import transforms
from torch.utils.data import DataLoader
from roboflow import Roboflow
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torch.optim as optim
from tqdm import tqdm



ModuleNotFoundError: No module named 'cv2'

In [ ]:
# Conexão com dataset de vidrarias de laboratório via Roboflow
rf = Roboflow(api_key="mpV6g84DqvN6HUqz9mt1")
project = rf.workspace("lab-identification").project("lab-identification-y2nq4")
version = project.version(11)
dataset = version.download("coco")  # formato compatível com PyTorch


In [ ]:
# Hiperparâmetros para treinamento
BATCH_SIZE = 2
IMAGE_SIZE = 640  # conforme o pré-processamento do dataset
EPOCHS = 10
LEARNING_RATE = 0.0001


In [ ]:
# Bloco para seleção de dispositivo (CUDA, MPS ou CPU)
if torch.cuda.is_available():
    # Prioridade 1: GPU NVIDIA (CUDA)
    device = torch.device("cuda")
    print("Dispositivo selecionado: GPU NVIDIA (CUDA)")

elif torch.backends.mps.is_available():
    # Prioridade 2: GPU Apple (MPS)
    device = torch.device("mps")
    print("Dispositivo selecionado: GPU Apple (MPS)")

else:
    # Fallback: CPU
    device = torch.device("cpu")
    print("Dispositivo selecionado: CPU")

print(f'Usando dispositivo: {device}')


In [ ]:
# Definir as transformações para os dados
transformacoes = transforms.Compose([
    transforms.ToTensor(),  # Converte para tensor e escala para [0, 1]
    #transforms.Normalize(mean=[0.485, 0.456, 0.406],  # Média dos canais RGB
                         #std=[0.229, 0.224, 0.225])   # Desvio padrão dos canais RGB
])




In [ ]:
import json
from collections import defaultdict

def preparar_anotacoes_coco(json_path):
    with open(json_path) as f:
        coco = json.load(f)

    # Mapear categorias para índices
    categories = coco["categories"]
    class_to_idx = {cat["name"]: cat["id"] for cat in categories}

    # Mapear ID da imagem para nome do arquivo
    id_to_filename = {img["id"]: img["file_name"] for img in coco["images"]}

    # Agrupar anotações por imagem
    grouped = defaultdict(lambda: {"boxes": [], "labels": []})
    for ann in coco["annotations"]:
        image_id = ann["image_id"]
        filename = id_to_filename[image_id]
        x, y, w, h = ann["bbox"]
        box = [x, y, x + w, y + h]
        label = ann["category_id"]

        grouped[filename]["boxes"].append(box)
        grouped[filename]["labels"].append(label)

    # Lista final de anotações
    annotations = [
        {
            "filename": fname,
            "boxes": grouped[fname]["boxes"],
            "labels": grouped[fname]["labels"]
        }
        for fname in grouped
    ]

    return annotations, class_to_idx

In [ ]:
# Classe personalizada
class LabGlasswareDataset(torch.utils.data.Dataset):
    def __init__(self, image_dir, annotations, transform=None):
        self.image_dir = image_dir
        self.annotations = annotations
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        ann = self.annotations[idx]
        img_path = os.path.join(self.image_dir, ann["filename"])
        image = Image.open(img_path).convert("RGB")

        boxes = torch.tensor(ann["boxes"], dtype=torch.float32)
        labels = torch.tensor(ann["labels"], dtype=torch.int64)

        target = {
            "boxes": boxes,
            "labels": labels
        }

        if self.transform:
            image = self.transform(image)

        return image, target



In [ ]:
# Diretórios
base_dir = "/content/LAB-IDENTIFICATION-11"
paths = {
    "train": os.path.join(base_dir, "train"),
    "valid": os.path.join(base_dir, "valid"),
    "test": os.path.join(base_dir, "test")
}


In [ ]:
# Anotações
annotations_train, class_to_idx = preparar_anotacoes_coco(os.path.join(paths["train"], "_annotations.coco.json"))
annotations_valid, _ = preparar_anotacoes_coco(os.path.join(paths["valid"], "_annotations.coco.json"))
annotations_test, _ = preparar_anotacoes_coco(os.path.join(paths["test"], "_annotations.coco.json"))

# Datasets
dataset_treino = LabGlasswareDataset(paths["train"], annotations_train, transformacoes)
dataset_validacao = LabGlasswareDataset(paths["valid"], annotations_valid, transformacoes)
dataset_teste = LabGlasswareDataset(paths["test"], annotations_test, transformacoes)

dataloader_treino = DataLoader(dataset_treino, batch_size=BATCH_SIZE, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))
dataloader_validacao = DataLoader(dataset_validacao, batch_size=BATCH_SIZE, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))
dataloader_teste = DataLoader(dataset_teste, batch_size=BATCH_SIZE, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))


In [ ]:
# DataLoaders
dataloader_treino = DataLoader(dataset_treino, batch_size=BATCH_SIZE, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))
dataloader_validacao = DataLoader(dataset_validacao, batch_size=BATCH_SIZE, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))
dataloader_teste = DataLoader(dataset_teste, batch_size=BATCH_SIZE, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))


In [ ]:
# Criar o modelo com número de classes personalizado

# Número de classes (inclui fundo = 0)
NUM_CLASSES = len(class_to_idx) + 1

# Carregar modelo pré-treinado
modelo = fasterrcnn_resnet50_fpn(pretrained=True)

# Substituir o cabeçote de classificação
in_features = modelo.roi_heads.box_predictor.cls_score.in_features
modelo.roi_heads.box_predictor = FastRCNNPredictor(in_features, NUM_CLASSES)

# Enviar para o dispositivo
modelo.to(device)


In [ ]:
# Definir otimizador e função de perda
import torch.optim as optim

params = [p for p in modelo.parameters() if p.requires_grad]
optimizer = optim.SGD(params, lr=LEARNING_RATE, momentum=0.9, weight_decay=0.0005)


In [ ]:
# Loop de treinamento

modelo.train()

for epoch in range(EPOCHS):
    print(f"\n🔁 Época {epoch+1}/{EPOCHS}")
    epoch_loss = 0.0

    # === LOOP DE TREINAMENTO ===
    for images, targets in tqdm(dataloader_treino, desc=f"Treinando época {epoch+1}"):
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        # Cálculo da perda
        loss_dict = modelo(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        # Acumular perda total
        epoch_loss += losses.item()

    # Média da perda da época
    epoch_loss /= len(dataloader_treino)
    print(f"📉 Loss médio de treino na época {epoch+1}: {epoch_loss:.4f}")

    # === LOOP DE VALIDAÇÃO ===
    modelo.eval()
    val_loss = 0.0

    with torch.no_grad():
        for images, targets in tqdm(dataloader_validacao, desc="Validando"):
            images = [img.to(device) for img in images]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            loss_dict = modelo(images, targets)
            if isinstance(loss_dict, dict):
                losses = sum(loss for loss in loss_dict.values())
                val_loss += losses.item()

    val_loss /= len(dataloader_validacao)
    print(f"✅ Loss médio de validação: {val_loss:.4f}")
    modelo.train()



In [ ]:
# Salvar o modelo treinado

torch.save(modelo.state_dict(), "modelo_labglassware.pth")
print("✅ Modelo salvo como modelo_labglassware.pth")
